In [91]:
import feather
import pickle
import numpy as np
import pandas as pd
from ml_metrics.average_precision import apk
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Train/Test split

In [ ]:
# Read song factors
with open('data/song_latent_factors.pkl', 'rb') as f:
    song_factors = pickle.load(f)

In [2]:
# Read relevant songs from embeddings and from spectrograms, and intersect them

rel_songs_from_embedding = feather.read_dataframe('data/embedding_songs_list.feather')
rel_songs_from_embedding = set([r_s.decode() for r_s in rel_songs_from_embedding['0']])

rel_songs_from_spectrograms = feather.read_dataframe('data/spectrogram_songs_list.feather')
rel_songs_from_spectrograms = set([r_s.decode() for r_s in rel_songs_from_spectrograms ['0']])

rel_songs = rel_songs_from_embedding.intersection(rel_songs_from_spectrograms)
rel_songs = [s for s in rel_songs if s in song_factors]

del rel_songs_from_embedding, rel_songs_from_spectrograms

In [3]:
train_songs,test_songs = train_test_split(rel_songs, test_size=0.2)

In [26]:
# Save train/test songs
labels = list(np.repeat('train', len(train_songs))) + list(np.repeat('test', len(test_songs)))
songs_train_test = pd.DataFrame({'song':train_songs+test_songs, 'label':labels})
feather.write_dataframe(songs_train_test, 'data/songs_train_test.feather')

del songs_train_test, train_songs

## Evaluation

In [7]:
# Read user factors
with open('data/user_latent_factors.pkl', 'rb') as f:
    user_factors = pickle.load(f)

In [171]:
# Keep only relevant song factors
song_factors = pd.DataFrame.from_dict({s:[song_factors[s]] for s in test_songs if s in song_factors}, orient='index')
song_factors.columns = ['factor']
song_factors.index.name = 'song'

In [31]:
# Create dictionary of user:songs (only for test songs)
user_songs = {user:set() for user in user_factors.keys()}
with open('./data/train_triplets.txt') as f:
    for line in f:
        s = line.strip().split('\t')
        user = users[s[0]]
        song = songs[s[1]]
        if song in song_factors.index:
            user_songs[user].add(song)

In [148]:
# Evaluation function
k = 500
def evaluate(user):
    scores = (song_factors['factor'] * user_factors[user]).apply(sum).sort_values(ascending=False)
    user_apk = apk(list(scores.index[:k]), user_songs[user])
    user_auc = roc_auc_score([s in user_songs[user] for s in scores.index], scores.values)
    return (user_apk,user_auc)

In [172]:
# Evaluate on all users
scores = [x for x in zip(*[evaluate(user) for user in user_songs.keys()])]

In [177]:
print('Mean average percision at {0}: {1:.4f}'.format(k, np.mean(scores[0])))
print('Mean area under ROC: {0:.4f}'.format(np.mean(scores[1])))

Mean average percision at 500: 0.6648
Mean area under ROC: 0.9104
